In [1]:
import git
from pathlib import Path
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import skew
import cv2
import warnings
from tqdm.notebook import tqdm
warnings.filterwarnings("ignore")
import pandas as pd
from collections import defaultdict
import re
import pickle

In [ ]:
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
nonskewed_df = pd.DataFrame(columns=['dataset', 'num_images', 'num_bootstrap', 'nonskewed_filter_idxs']).set_index(['dataset', 'num_images', 'num_bootstrap'])
nonskewed_df.to_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv'))
nonskewed_df

,,,nonskewed_filter_idxs
dataset,num_images,num_bootstrap,


In [ ]:
np.random.seed(0)

# Load pretrained AlexNet
alexnet = torchvision.models.alexnet(pretrained=True)
alexnet.eval()  # set to evaluation mode

# Extract the first convolutional layer filters
first_conv = alexnet.features[0]
filters = first_conv.weight.data.clone().cpu().numpy()  # shape: [out_channels, in_channels, height, width]

# pd.to_pickle(filters, os.path.join(ROOT_DIR, 'learned-filters', 'filters.pickle'))

filter_groups = {
    "single_edge": [3, 6, 10, 11, 12, 13, 14, 23, 24, 28, 29, 30, 32, 34, 43, 48, 49, 50, 55, 57], #gabor-like / edge detector
    "multi_edge": [9, 16, 18, 22, 25, 27, 33, 41, 54, 63], #complex gabor / complex edge detector
    "eye": [21, 31, 37, 39, 45, 46,], # color contrast
    "dual_color": [0, 2, 4, 5, 17, 20, 26, 38, 42, 44, 47, 56, 59], # color contrast
    "inside_out": [7, 15, 19, 35, 40, 51, 52, 53, 58], # smoothing
    "misc": [1, 8, 36, 60, 61, 62] # misc
}

In [ ]:
filter_group_map = {}
for group, indices in filter_groups.items():
    for index in indices:
        filter_group_map[index] = group
pd.to_pickle(filter_group_map, os.path.join(ROOT_DIR, 'learned-filters', 'filter_group_map.pickle'))
filter_group_map